# 1) Convert PDF to JPG 

In [2]:
import glob
from pdf2image import convert_from_path
import os
from PIL import Image

# ---------------------------------------------------------------------------

input_path = 'pdf_forms/'
output_path = 'jpg_forms/raw_images/'

# ---------------------------------------------------------------------------

!mkdir jpg_forms
!mkdir jpg_forms/raw_images

# ---------------------------------------------------------------------------

pdf_list = glob.glob1(input_path,"*.pdf")
print("There are",len(pdf_list),"pdf files")

# ---------------------------------------------------------------------------

from tqdm import tqdm
j = 0
for i in tqdm(range(len(pdf_list))):
    pages = convert_from_path(input_path+pdf_list[i])
    for page in pages:
        page.save(output_path + 'image_' + str(j) + ".jpg", 'JPEG')
        j+=1
        
print("Successfully converted to jpg images. See the output at",output_path)

# ---------------------------------------------------------------------------

jpg_list = glob.glob1(output_path,"*.jpg")
print("\nThere are", len(jpg_list), "images to be labelled")

# ---------------------------------------------------------------------------

  0%|          | 0/104 [00:00<?, ?it/s]

There are 104 pdf files


100%|██████████| 104/104 [01:34<00:00,  1.10it/s]

Successfully converted to jpg images. See the output at jpg_forms/raw_images/

There are 400 images to be labelled


# 2) Resize JPG dimensions to 600 x 600

In [3]:
from PIL import Image
jpg_list = glob.glob1(output_path,"*.jpg")
output_path2 = 'jpg_forms/resized_images/'

!mkdir jpg_forms/resized_images

for i in tqdm(range(len(jpg_list))):
    img = Image.open(output_path+jpg_list[i])
    img = img.resize((430,600))
    img.save(output_path2 + 'resized_image_' + str(i) + ".jpg", 'JPEG')

100%|██████████| 400/400 [00:29<00:00, 13.50it/s]


# 3) Label objects in images using labelImg

In [20]:
!python3 ../labelImg/labelImg.py

../labelImg/labelImg.py:203: DeprecationWarning: an integer is required (got type DockWidgetFeatures).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  self.dock.setFeatures(self.dock.features() ^ self.dockFeatures)
../labelImg/labelImg.py:950: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  bar.setValue(bar.value() + bar.singleStep() * units)
../labelImg/labelImg.py:956: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  self.zoomWidget.setValue(value)
../labelImg/labelImg.py:1010: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  h_bar.setValue(new_h_

# 4) Convert XML to CSV files

In [23]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(input_path, output_path):
    xml_list = []
    for xml_file in glob.glob(input_path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    xml_df.to_csv(output_path + 'labels.csv', index=None)
    return xml_df

xml_to_csv(output_path2 , 'jpg_forms/selected_images/')

,filename,width,height,class,xmin,ymin,xmax,ymax
0,resized_image_15.jpg,430,600,Name,26,226,416,241
1,resized_image_15.jpg,430,600,Email,26,262,216,282
2,resized_image_15.jpg,430,600,Type_of_Form,26,168,201,177
3,resized_image_2.jpg,430,600,Logo,345,20,410,40
4,resized_image_2.jpg,430,600,Logo,14,563,89,585
...,...,...,...,...,...,...,...,...
83,resized_image_31.jpg,430,600,Logo,19,18,137,46
84,resized_image_31.jpg,430,600,Type_of_Form,19,58,278,93
85,resized_image_31.jpg,430,600,Name,20,122,267,140
86,resized_image_31.jpg,430,600,IC,272,122,413,140
